## Womanium Assignment 6.12

### Objective: creating a quantum walk operator for the case of a line with 16 nodes

In [1]:
from classiq import *

In [2]:
# Define the number of qubits
num_qubits = 4
num_vertices = 2**num_qubits

### Phase kickback

In [3]:
# Prepare a qubit in the minus state
@qfunc
def prepare_minus(x: QBit):
    X(x)
    H(x)

# Flip the phase of auxiliary qubit if the vertice is not zero
@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum):
    aux^=(x!=0)

# Apply the Grover diffusion operator
@qfunc
def zero_diffuzer(x: QNum):
    aux = QNum('aux')
    allocate(1,aux)
    within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)


### Walk operator

In [4]:
# Define the movements of nodes for each iteration
def W_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):
    # Initialisation
    prob = [0 for j in range(num_vertices)]
    # Control the movement of the first node (can only move to the second node)
    if i == 0:
        prob[i + 1] = 1.0
    
    # Control the movement of the last node (can only move to the second to last node)
    elif i == num_vertices - 1:
        prob[i - 1] = 1.0

    # Control the movement of middle nodes
    else:
        prob[i - 1] = 0.5  # Probability of moving to the left node
        prob[i + 1] = 0.5  # Probability of moving to the right node
    
    print(f'Node={i}, prob vec ={prob}')

    # Apply phase kickback
    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute=lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action=lambda: zero_diffuzer(adjacent_vertices)))

# Iterate over all nodes
@qfunc 
def W_operator(vertices:QNum, adjacent_vertices: QNum):

    for i in range(num_vertices):
        W_iteration(i,vertices,adjacent_vertices)

### Shift operator

In [5]:
# Check if two nodes are adjacent 
@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
    res |= (((vertices+adjacent_vertices)%2) ==1)

# Swap two states bit by bit
@qfunc 
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
    repeat(count= x.len,
           iteration= lambda i: SWAP(x[i],y[i])) 

# Change the vertices' number
@qfunc 
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))

### Perform the quantum random walk

In [6]:
@qfunc 
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):
    # Initialise qubits representing vertices
    allocate(num_qubits, vertices)

    # Create superposition of all states ∣0⟩, ∣1⟩,..., ∣15⟩
    hadamard_transform(vertices)

    # Initialise qubits representing adjacent vertices
    allocate(num_qubits, adjacent_vertices)
    
    # Apply walk operator and shift operator
    W_operator(vertices, adjacent_vertices)
    S_operator(vertices, adjacent_vertices)

### Create and synthesise the circuit

In [ ]:
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)